## This notebook will create Excel files based on model performance(best R2 value) for top 10 sodas in dataset.The file names are created based on soda product names in data. Also for reference it will create a CSV file ('model_performance.csv') with all metric values (R2, MAE, MSE, RMSE)calculated for all models.

In [ ]:
# !pip install lightgbm
# !pip install --user xgboost

In [125]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
import category_encoders as ce
import math
import xgboost as xgb
import lightgbm


import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.losses import MeanSquaredLogarithmicError
from sklearn.tree import DecisionTreeRegressor

In [91]:
# loading data from csv file
df1 = pd.read_csv('data_2023_sodas.csv')
df2 = pd.read_csv('data_2022_sodas.csv')
# combining dataframes
frames = [df1, df2]
df = pd.concat(frames)
df.shape

/Users/farhanaislam/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(656551, 24)

In [92]:
# identifying top sold sodas
df.groupby(['SaleDescription'])['SaleDescription'].count().sort_values(ascending=False)

SaleDescription
COCA COLA REGULAR TRAY 24X330ML    115255
FANTA EXOTIC 330ML                 101404
FANTA STRAWBERRY KIWI 330ML         77750
COCA COLA REGULAR 330ML             58995
FANTA ELDERFLOWER 330ML             58140
FANTA CASSIS 330ML                  51620
COCA COLA VANILLE (D) 330ML         51591
COCA COLA CHERRY BLIK 330ML         49765
FANTA LEMON 330ML                   47822
COCA COLA ZERO TRAY 24X330ML        44209
Name: SaleDescription, dtype: int64

In [93]:
# adding day number (1 to 30 per month) and week number (1 to 52 weeks per year)
df['Date1'] = pd.to_datetime(df['Day'],errors ='coerce')
# df['Week_num'] = df['Date1'].dt.week
df['Day_num'] = df['Date1'].dt.day

In [94]:
# function to replace null values in df
def replace_null(df):
    df['Holiday'].fillna('Not_holiday', inplace = True)
    df['SchoolHoliday'].fillna('Not_school_holiday', inplace = True)
    return df

In [102]:
def data_preprocessing(df,item):
    # creating dataframe with only supplied item
    df1 = df[df['SaleDescription']== item]
    
    # Adding total num of sales per day as target column
    Num_sales = df1.groupby(['Date1'], as_index=False)['SaleDescription'].count()

    
    #group by data according to per day sale and adding target column to dataframe
    dummydb = df1.groupby('Date1', as_index=False).last()
    dummydb['Num_sale'] = Num_sales['SaleDescription']
    
    # replacing all null values
    replace_null(dummydb)
    
    # Converting day_num, Month, year, weekday,Weekend, holidayBool, rainBool as categorical column
    dummydb['Month'] = dummydb['Month'].map(str)
    dummydb['Year'] = dummydb['Year'].map(str)
    dummydb['Weekday'] = dummydb['Weekday'].map(str)
    dummydb['HolidayBool'] = dummydb['HolidayBool'].map(str)
    dummydb['RainBool'] = dummydb['RainBool'].map(str)
    dummydb['Day_num'] = dummydb['Day_num'].map(str)
    dummydb['Weekend'] = dummydb['Weekend'].map(str)
    
    #considering the following columns as features for model
    pre_processed_df = dummydb[['Weekday','Day_num', 'Month', 'Year', 'Weekend', 'Season', 'Holiday', 'SchoolHoliday', 'HolidayBool', 'Avg Wind Speed'\
               , 'Avg Temp', 'Min Temp', 'Max Temp', 'Sun Hours', 'Hours of rain', 'Rain','RainBool', 'Num_sale','Date1','Day']]
    
    return pre_processed_df


In [116]:
def train_test_df_for_XGBoost_lightGBM(train_df,test_df):
    X_train = train_df.drop(['Num_sale','Date1','Day'], axis = 1)
    y_train = train_df[['Num_sale']]
    X_test = test_df.drop(['Num_sale','Date1','Day'], axis = 1)
    y_test = test_df[['Num_sale']]
    
    # Extract text features
    cats = list(X_train.loc[:,X_train.dtypes == 'object'].columns.values)

    # Convert to Pandas category
    for col in cats:
        X_train[col] = X_train[col].astype('category')
        X_test[col] = X_test[col].astype('category')
        
    return X_train,y_train,X_test,y_test

In [117]:
def train_test_df_for_NN(train_df,test_df):
    train_df_1 = train_df.drop(['Day'],axis = 1)
    test_df_1 = test_df.drop(['Day'],axis = 1)
    col_list = train_df_1.select_dtypes(include=['object']).columns.to_list()

    encoder =  ce.BinaryEncoder(cols = col_list, return_df = True )
    train_df_encoded =  encoder.fit_transform(train_df_1)
    test_df_encoded = encoder.transform(test_df_1)
    
    X_train1 = train_df_encoded.drop(['Num_sale','Date1'], axis = 1)
    y_train1 = train_df_encoded[['Num_sale']]
    X_test1 = test_df_encoded.drop(['Num_sale', 'Date1'], axis = 1)
    y_test1 = test_df_encoded[['Num_sale']]
    
    return X_train1,y_train1,X_test1,y_test1

In [127]:
def model_Decision_tree(X_train,y_train,X_test,y_test):
    model_DT = DecisionTreeRegressor(random_state = 42)
    model_DT.fit(X_train, y_train)
    DT_predictions = model_DT.predict(X_test)
    
    r2_DT = r2_score(y_test, DT_predictions)
#     print(f'R2 value XGBoost: {r2_XGB}')
    MSE = mean_squared_error(y_test, DT_predictions)
#     print(f'Mean Square Error XGBoost: {MSE}')
    RMSE = math.sqrt(MSE)
#     print(f'Root Mean Square Error XGBoost: {RMSE}')
    MAE = mean_absolute_error(y_test,DT_predictions)
    
    return r2_DT,MSE,RMSE,DT_predictions,MAE

In [118]:
def model_XGBoost(X_train,y_train,X_test,y_test):
    model_XGB = xgb.XGBRegressor(enable_categorical=True, tree_method= 'hist',gamma = 1, random_state = 101, learning_rate = 0.001, max_depth = 30, early_stopping_rounds = 10)
    model_XGB.fit(X_train, y_train, eval_set=[(X_test, y_test)])
    
    XGB_predictions = model_XGB.predict(X_test)
    
    r2_XGB = r2_score(y_test, XGB_predictions)
#     print(f'R2 value XGBoost: {r2_XGB}')
    MSE = mean_squared_error(y_test, XGB_predictions)
#     print(f'Mean Square Error XGBoost: {MSE}')
    RMSE = math.sqrt(MSE)
#     print(f'Root Mean Square Error XGBoost: {RMSE}')
    MAE = mean_absolute_error(y_test,XGB_predictions)
    
#     test_df['predictions_XGB'] = XGB_predictions
#     output_XGB = test_df[['Date1','Num_sale','predictions_XGB']]

#     feature_importance = model.feature_importances_
#     sorted_idx = np.argsort(feature_importance)
#     fig = plt.figure(figsize=(12, 6))
#     plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align='center')
#     plt.yticks(range(len(sorted_idx)), np.array(X_test.columns)[sorted_idx])
#     plt.title('XGBoost Feature Importance')
    
    return r2_XGB, MSE, RMSE, XGB_predictions, MAE
    
    

In [119]:
def model_LightGBM(X_train,y_train,X_test,y_test):
    model_lgbm = lightgbm.LGBMRegressor()
    model_lgbm.fit(X_train, y_train)
    
    LGBM_predictions = model_lgbm.predict(X_test)
    
    r2_LGBM = r2_score(y_test, LGBM_predictions)
#     print(f'R2 value LightGBM: {r2_LGBM}')
    MSE = mean_squared_error(y_test, LGBM_predictions)
#     print(f'Mean Square Error LightGBM: {MSE}')
    RMSE = math.sqrt(MSE)
#     print(f'Root Mean Square Error LightGBM: {RMSE}')
    MAE = mean_absolute_error(y_test,LGBM_predictions)
    
#     test_df['predictions_LGBM'] = LGBM_predictions
#     output_LGBM = test_df[['Day','Num_sale','predictions_LGBM']]
#     lightgbm.plot_importance(lgbmr)
    
    return r2_LGBM, MSE, RMSE, LGBM_predictions, MAE

In [123]:
def model_NN(X_train1,y_train1,X_test1,y_test1):
    hidden_units1 = 10
    hidden_units2 = 10
    hidden_units3 = 10
    learning_rate = 0.01
    # Creating model using the Sequential in tensorflow
    def build_model_using_sequential():
        model = Sequential([
        Dense(hidden_units1, kernel_initializer='normal', activation='relu'),
        Dropout(0.2),
        Dense(hidden_units2, kernel_initializer='normal', activation='relu'),
        Dropout(0.2),
        Dense(hidden_units3, kernel_initializer='normal', activation='relu'),
        Dense(1, kernel_initializer='normal', activation='linear')
          ])
        return model
    
    # build the model
    model_NN = build_model_using_sequential()
    # loss function
    msle = MeanSquaredLogarithmicError()
    model_NN.compile(
    loss=msle, 
    optimizer=Adam(learning_rate=learning_rate), 
    metrics=[msle]
    )
    # train the model
    history = model_NN.fit(
        X_train1.values, 
        y_train1.values, 
        epochs=150, 
        batch_size=16,
        validation_split=0.2
    )
    y_pred_NN = model_NN.predict(X_test1)
    r2_NN = r2_score(y_test1, y_pred_NN)
#     print(f'r2 score :{r2_NN}')

    MSE = mean_squared_error(y_test1, y_pred_NN)

    RMSE = math.sqrt(MSE)
#     print(f'Root Mean Square Error: {RMSE}')
    MAE = mean_absolute_error(y_test1,y_pred_NN)
    
#     test_df['predict_NN'] = y_pred_NN
#     Output_NN = test_df[['Day','Num_sale','predict_NN']]
    
    return r2_NN, MSE, RMSE, y_pred_NN, MAE

In [101]:
list_soda = df['SaleDescription'].unique()
print(list_soda)

['COCA COLA REGULAR 330ML' 'COCA COLA REGULAR TRAY 24X330ML'
 'COCA COLA ZERO TRAY 24X330ML' 'FANTA ELDERFLOWER 330ML'
 'COCA COLA VANILLE (D) 330ML' 'FANTA CASSIS 330ML' 'FANTA EXOTIC 330ML'
 'FANTA STRAWBERRY KIWI 330ML' 'FANTA LEMON 330ML'
 'COCA COLA CHERRY BLIK 330ML']


In [132]:
def make_prediction(df):
    list_of_resluts = []
    list_soda = df['SaleDescription'].unique()
    
    for item in list_soda:

        pre_processed_df = data_preprocessing(df,item)
        # we can not use train test split in this dataframe, because we have time series data. 
        # So we considered all data before 30-05-2023 as training data and the rest as test
        train_df = pre_processed_df[pre_processed_df['Date1']<'2023-05-30']
        test_df = pre_processed_df[pre_processed_df['Date1']>'2023-05-30']
        print(item, train_df.shape, test_df.shape)

        # implementing XGBoost
#         X_train,y_train,X_test,y_test = train_test_df_for_XGBoost_lightGBM(train_df,test_df)
#         r2_XGB, MSE_XGB, RMSE_XGB, output_XGB, MAE_XGB = model_XGBoost(X_train,y_train,X_test,y_test)
#         list_of_resluts.append([item,'XGBoost', r2_XGB, MSE_XGB, RMSE_XGB, MAE_XGB])
        
        
        # implementing Decision_Tree
        X_train,y_train,X_test,y_test = train_test_df_for_NN(train_df,test_df)
        r2_XGB, MSE_XGB, RMSE_XGB, output_XGB, MAE_XGB= model_Decision_tree(X_train,y_train,X_test,y_test)
#         list_of_resluts.append([item,'Decision_Tree', r2_XGB, MSE_XGB, RMSE_XGB, MAE_XGB])
        list_of_resluts.append([item,'Decision_Tree', round(r2_XGB,4), round(MSE_XGB,4), round(RMSE_XGB,4),  round(MAE_XGB,4)]) 
        
        # implementing LightGBM
        X_train,y_train,X_test,y_test = train_test_df_for_XGBoost_lightGBM(train_df,test_df)
        r2_LGBM, MSE_LGBM, RMSE_LGBM, output_LGBM, MAE_LGBM = model_LightGBM(X_train,y_train,X_test,y_test)
        list_of_resluts.append([item,'LightGBM', round(r2_LGBM,4), round(MSE_LGBM,4), round(RMSE_LGBM,4), round(MAE_LGBM,4)]) 
    
        #implementing Neural network
        X_train1,y_train1,X_test1,y_test1 = train_test_df_for_NN(train_df,test_df)
        r2_NN, MSE_NN, RMSE_NN, Output_NN, MAE_NN = model_NN(X_train1,y_train1,X_test1,y_test1)
        list_of_resluts.append([item,'NN', round(r2_NN,4), round(MSE_NN,4), round(RMSE_NN,4),  round(MAE_NN,4)]) 
    
        if (r2_XGB > r2_LGBM and r2_XGB >r2_NN):
            print(f'Best model for {item} is XGBoost R2 value : {r2_XGB}, MSE: {MSE_XGB}, RMSE: {RMSE_XGB}')
            test_df['predictions_XGB'] = output_XGB
            output = test_df[['Day','Num_sale','predictions_XGB']]

        elif (r2_LGBM > r2_XGB and r2_LGBM > r2_NN ): 
            print(f'Best model for {item} is LightGBM R2 value : {r2_LGBM}, MSE: {MSE_LGBM}, RMSE: {RMSE_LGBM}') 
            test_df['predictions_LGBM'] = output_LGBM
            output = test_df[['Day','Num_sale','predictions_LGBM']]

        else :
            print(f'Best model for {item} is NN R2 value : {r2_NN}, MSE: {MSE_NN}, RMSE: {RMSE_NN}') 
            test_df['predict_NN'] = Output_NN
            output = test_df[['Day','Num_sale','predict_NN']]
        # dynamically allocating file name
        filename = f'{item}.xlsx'

        output.to_excel(filename, index = False)
        
    return list_of_resluts
    

In [133]:
def create_output_files(df):
    list_of_resluts = make_prediction(df)
    # Create an empty DataFrame with column names
    model_performance = pd.DataFrame(list_of_resluts, columns=['Product_Name', 'Model_name', 'R2_score','MSE','RMSE','MAE'])
    model_performance.to_csv('model_performance.csv', index=False)

create_output_files(df)

COCA COLA REGULAR 330ML (210, 20) (31, 20)
Epoch 1/150
11/11 [==============================] - 0s 11ms/step - loss: 23.4578 - mean_squared_logarithmic_error: 23.1320 - val_loss: 9.6146 - val_mean_squared_logarithmic_error: 9.7248
Epoch 2/150
11/11 [==============================] - 0s 3ms/step - loss: 5.7555 - mean_squared_logarithmic_error: 5.6029 - val_loss: 0.5959 - val_mean_squared_logarithmic_error: 0.5991
Epoch 3/150
11/11 [==============================] - 0s 3ms/step - loss: 1.0541 - mean_squared_logarithmic_error: 1.0496 - val_loss: 1.0030 - val_mean_squared_logarithmic_error: 0.9700
Epoch 4/150
11/11 [==============================] - 0s 3ms/step - loss: 0.9959 - mean_squared_logarithmic_error: 0.9842 - val_loss: 1.3162 - val_mean_squared_logarithmic_error: 1.2772
Epoch 5/150
11/11 [==============================] - 0s 3ms/step - loss: 0.9441 - mean_squared_logarithmic_error: 0.9317 - val_loss: 1.0599 - val_mean_squared_logarithmic_error: 1.0270
Epoch 6/150
11/11 [==========

<ipython-input-132-740827a68b3d>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['predict_NN'] = Output_NN


COCA COLA REGULAR TRAY 24X330ML (375, 20) (31, 20)
Epoch 1/150
19/19 [==============================] - 0s 7ms/step - loss: 15.5132 - mean_squared_logarithmic_error: 15.3498 - val_loss: 2.0574 - val_mean_squared_logarithmic_error: 2.0328
Epoch 2/150
19/19 [==============================] - 0s 2ms/step - loss: 1.2494 - mean_squared_logarithmic_error: 1.2553 - val_loss: 0.5561 - val_mean_squared_logarithmic_error: 0.5653
Epoch 3/150
19/19 [==============================] - 0s 2ms/step - loss: 1.0974 - mean_squared_logarithmic_error: 1.0994 - val_loss: 0.4198 - val_mean_squared_logarithmic_error: 0.4234
Epoch 4/150
19/19 [==============================] - 0s 2ms/step - loss: 0.9106 - mean_squared_logarithmic_error: 0.9086 - val_loss: 0.4220 - val_mean_squared_logarithmic_error: 0.4216
Epoch 5/150
19/19 [==============================] - 0s 2ms/step - loss: 0.8102 - mean_squared_logarithmic_error: 0.8093 - val_loss: 0.4552 - val_mean_squared_logarithmic_error: 0.4531
Epoch 6/150
19/19 [===

<ipython-input-132-740827a68b3d>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['predict_NN'] = Output_NN


Epoch 1/150
19/19 [==============================] - 0s 6ms/step - loss: 10.9551 - mean_squared_logarithmic_error: 10.8375 - val_loss: 0.5305 - val_mean_squared_logarithmic_error: 0.5160
Epoch 2/150
19/19 [==============================] - 0s 2ms/step - loss: 1.1468 - mean_squared_logarithmic_error: 1.1388 - val_loss: 0.6989 - val_mean_squared_logarithmic_error: 0.7050
Epoch 3/150
19/19 [==============================] - 0s 2ms/step - loss: 0.9693 - mean_squared_logarithmic_error: 0.9686 - val_loss: 0.3675 - val_mean_squared_logarithmic_error: 0.3620
Epoch 4/150
19/19 [==============================] - 0s 2ms/step - loss: 0.7307 - mean_squared_logarithmic_error: 0.7332 - val_loss: 0.3712 - val_mean_squared_logarithmic_error: 0.3643
Epoch 5/150
19/19 [==============================] - 0s 2ms/step - loss: 0.6949 - mean_squared_logarithmic_error: 0.6987 - val_loss: 0.4148 - val_mean_squared_logarithmic_error: 0.4040
Epoch 6/150
19/19 [==============================] - 0s 2ms/step - loss: 

...

<ipython-input-132-740827a68b3d>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['predictions_LGBM'] = output_LGBM


Epoch 1/150
17/17 [==============================] - 0s 7ms/step - loss: 10.7177 - mean_squared_logarithmic_error: 10.2707 - val_loss: 2.2203 - val_mean_squared_logarithmic_error: 2.0980
Epoch 2/150
17/17 [==============================] - 0s 2ms/step - loss: 1.8854 - mean_squared_logarithmic_error: 1.8263 - val_loss: 3.3370 - val_mean_squared_logarithmic_error: 2.7272
Epoch 3/150
17/17 [==============================] - 0s 2ms/step - loss: 1.7649 - mean_squared_logarithmic_error: 1.7455 - val_loss: 2.3782 - val_mean_squared_logarithmic_error: 2.0816
Epoch 4/150
17/17 [==============================] - 0s 2ms/step - loss: 1.3671 - mean_squared_logarithmic_error: 1.3933 - val_loss: 2.1812 - val_mean_squared_logarithmic_error: 1.9846
Epoch 5/150
17/17 [==============================] - 0s 2ms/step - loss: 1.2647 - mean_squared_logarithmic_error: 1.2228 - val_loss: 2.1434 - val_mean_squared_logarithmic_error: 1.9639
Epoch 6/150
17/17 [==============================] - 0s 2ms/step - loss: 

<ipython-input-132-740827a68b3d>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['predictions_LGBM'] = output_LGBM


Epoch 1/150
17/17 [==============================] - 1s 8ms/step - loss: 20.5380 - mean_squared_logarithmic_error: 20.6480 - val_loss: 24.2839 - val_mean_squared_logarithmic_error: 25.6260
Epoch 2/150
17/17 [==============================] - 0s 2ms/step - loss: 20.5380 - mean_squared_logarithmic_error: 20.5616 - val_loss: 24.2839 - val_mean_squared_logarithmic_error: 25.6260
Epoch 3/150
17/17 [==============================] - 0s 2ms/step - loss: 20.5379 - mean_squared_logarithmic_error: 20.6875 - val_loss: 24.2839 - val_mean_squared_logarithmic_error: 25.6260
Epoch 4/150
17/17 [==============================] - 0s 2ms/step - loss: 10.8651 - mean_squared_logarithmic_error: 10.5695 - val_loss: 0.6009 - val_mean_squared_logarithmic_error: 0.5319
Epoch 5/150
17/17 [==============================] - 0s 2ms/step - loss: 2.1258 - mean_squared_logarithmic_error: 2.1586 - val_loss: 0.9219 - val_mean_squared_logarithmic_error: 0.7811
Epoch 6/150
17/17 [==============================] - 0s 2ms/s

<ipython-input-132-740827a68b3d>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['predict_NN'] = Output_NN
